In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline
import ipywidgets
from funciones import *
from ipywidgets import fixed
from functools import partial
from IPython.display import display

# backends: notebook, inline, ipympl...


In [3]:

puntos_rutina_num = [64,128,256,512]


puntos_num_bandas = np.arange(6,16,1)
#resultados_num_bandas = rutina(num_bandas=6,
#       archivo_control='control-num1.ctl',
#       control=[6,64,3], # num_bandas, resolucion, mesh;  e,a; e max 
#       puntos=puntos_num_bandas,
#       tipo=['num',0], # num, mat o lineal. indice para introduccion   
#       cifras=2, # solo se usa para el balance de proporciones
#       nombre='rutina-num-bandas',
#       funcion=default
#      )

puntos_resolucion = [2**(i+4) for i in range(10)]
#resultados_resolucion = rutina(num_bandas=6,
#       archivo_control='control-num1.ctl',
#       control=[6,64,3], # num_bandas, resolucion, mesh;  e,a; e max 
#       puntos=puntos_resolucion,
#       tipo=['num',1], # num, mat o lineal. indice para introduccion   
#       cifras=2, # solo se usa para el balance de proporciones
#       nombre='rutina-resolucion',
#       funcion=default
#      )

puntos_mesh = np.arange(5, 500, 50)
#resultados_mesh = rutina(num_bandas=6,
#       archivo_control='control-num1.ctl',
#       control=[6,64,3], # num_bandas, resolucion, mesh;  e,a; e max 
#       puntos=puntos_mesh,
#       tipo=['num',2], # num, mat o lineal. indice para introduccion   
#       cifras=2, # solo se usa para el balance de proporciones
#       nombre='rutina-mesh',
#       funcion=default
#      )

#anhadir interpolacion?

puntos_epsilon_a = np.round(np.arange(1.5,6.5,0.5),2)


In [4]:
nombre = 'epsilon-a'
with open('resultados-' + nombre + '.pkl', 'rb') as f:
    resultados = pickle.load(f)

nombre = 'epsilon-a-mesh'
with open('resultados-' + nombre + '.pkl', 'rb') as f:
    resultados_mesh = pickle.load(f)
    
print(np.max(resultados['tefreqs'][0] - resultados_mesh['tefreqs'][0]))


1.0000000000509601e-05


In [5]:
texto = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                         str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
            'vels' : { 'titulo' : lambda track,pola,identidad: 
                                         "Velocidades de grupo para " + str(track+1) + " bandas "
                                          + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
          'rutina' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'Puntos de la rutina (u.i)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
         'widgets' : { 'puntos' : lambda _: 'Punto de rutina',
                      'num_gap' : lambda _: 'Gap',
                        'serie' : lambda _: 'Serie:'
                     }
        }


In [6]:
def traductor(param_red, resultados):
    c = 3.0e8
    factor = c/param_red

    for i in range(len(resultados['tefreqs'])):
        resultados['tefreqs'][i] = factor*resultados['tefreqs'][i]
    
    resultados['gaps']['Ancho TE'] = factor*resultados['gaps']['Ancho TE']
    resultados['gaps']['Prom TE'] = factor*resultados['gaps']['Prom TE']

    return resultados


def graficador(resultados,
               puntos=0,
               num_bandas=6,
               pola='te',
               texto=0,
               param_red=0.0,
               rango=0.0,
               mostrar_velocidades=False
              ):
    
    if isinstance(texto, int):
        texto = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                             str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                                 'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                                 'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                             },
                    'vels' : { 'titulo' : lambda track,pola,identidad: 
                                             "Velocidades de grupo para " + str(track+1) + " bandas "
                                             + pola.upper() + " con " + identidad,
                                 'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                                 'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                             },
                  'rutina' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                             + ' en función de los puntos de la rutina', 
                                 'ejex' : lambda _: 'Puntos de la rutina (u.i)',
                                 'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                             },
                 'widgets' : { 'puntos' : lambda _: 'Punto de rutina',
                              'num_gap' : lambda _: 'Gap',
                                'serie' : lambda _: 'Serie:'
                     }
                }
    
    if (pola=='te'):
        llaves = ['tefreqs', 'tevels',  'Ancho TE', 'Promedio TE', 'Porcentaje TE']
    elif (pola=='tm'):
        llaves = ['tmfreqs', 'tmvels',  'Ancho TM', 'Promedio TM', 'Porcentaje TM']
    
    try:
        with open('resultados-' + resultados + '.pkl', 'rb') as f:
            resultados = pickle.load(f)
    except:
        try:
            with open(resultados + '/resultados-' + resultados + '.pkl', 'rb') as f:
                resultados = pickle.load(f)
        except:
            pass
    
    if isinstance(puntos, int): puntos = resultados['puntos']
    
    if (param_red != 0.0):
        resultados = traductor(param_red, resultados)
    
    def func_indice(valor): 
        try:
            indice = int(np.where(puntos == valor)[0][0])
        except:
            indice = puntos.index(valor)
        return indice
    
    def graf_bandas(valor, num_bandas, track=num_bandas-1):
        indice = func_indice(valor)
        fig, ax = plt.subplots()
        for i in range(track+1):
            k_vector = np.linspace(-0.5, 0.5, int(len(resultados[llaves[0]][indice][i])))
            ax.scatter(k_vector, resultados[llaves[0]][indice][i], c='r', s=0.1)
        
        parametros = resultados['parametros'][indice]
        identidad = str(parametros[0])
        for i in range(len(parametros)-1):
            identidad = identidad + '-' + str(parametros[i+1])

        ax.set_xlabel(texto['bandas']['ejex'](0))
        ax.set_ylabel(texto['bandas']['ejey'](0))
        ax.set_title(texto['bandas']['titulo'](track,pola,identidad))
        
        if (rango != 0.0):
            ax.set_ylim(rango)
        
    def graf_vels(valor, num_bandas, track=num_bandas-1):
        indice = func_indice(valor)
        fig, ax = plt.subplots()
        for i in range(track+1):
            k_vector = np.linspace(-0.5, 0.5, int(len(resultados[llaves[1]][indice][i])))
            ax.scatter(k_vector, resultados[llaves[1]][indice][i], c='r', s=0.1)
        
        parametros = resultados['parametros'][indice]
        identidad = str(parametros[0])
        for i in range(len(parametros)-1):
            identidad = identidad + '-' + str(parametros[i+1])

        ax.set_xlabel(texto['vels']['ejex'](0))
        ax.set_ylabel(texto['vels']['ejey'](0))
        ax.set_title(texto['vels']['titulo'](track,pola,identidad))
        
        if (rango != 0.0):
            ax.set_ylim(rango)
        
    def graf_rutina(num_gap=1,tipo=llaves[2]):
        fig, ax = plt.subplots()
        fig.subplots_adjust(0.2,0.2)
        plt.ticklabel_format(style='plain', useOffset=False)
        ax.scatter(puntos, resultados['gaps'][tipo][num_gap-1], c='r', s=1)
        ax.set_xlabel(texto['rutina']['ejex'](0))
        ax.set_ylabel(texto['rutina']['ejey'](num_gap))
        if (tipo==llaves[2]):
            ax.set_title("Ancho " + texto['rutina']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        elif (tipo==llaves[3]):
            ax.set_title("Promedio " + texto['rutina']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        else:
            ax.set_title("Porcentaje " + texto['rutina']['titulo'](num_gap))
        
    
    ipywidgets.interact(graf_bandas,
                        valor=ipywidgets.SelectionSlider(description=texto['widgets']['puntos'](0),
                                                         options=puntos
                                                        ),
                        num_bandas=fixed(num_bandas),
                        track=fixed(num_bandas-1)
                       )
    
    if (mostrar_velocidades == True):
        ipywidgets.interact(graf_vels,
                            valor=ipywidgets.SelectionSlider(description=texto['widgets']['puntos'](0),
                                                             options=puntos
                                                            ),
                            num_bandas=fixed(num_bandas),
                            track=fixed(num_bandas-1)
                           )
    
    ipywidgets.interact(graf_rutina,
                        num_gap=ipywidgets.SelectionSlider(description=texto['widgets']['num_gap'](0),
                                                         options=np.arange(1,num_bandas,1)
                                                        ),
                        tipo=ipywidgets.Dropdown(options=[llaves[2], llaves[3], llaves[4]],
                                             value=llaves[2],
                                             description=texto['widgets']['serie'](0),
                                             disabled=False,
                                            )
                       )
#    return resultados

In [7]:
#resultados = graficador('punto-2.41-1.0-0.81-0.19')


In [8]:
graficador('aprox-lineal-2mat')

interactive(children=(SelectionSlider(description='Punto de rutina', options=(1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.…

interactive(children=(SelectionSlider(description='Gap', options=(1, 2, 3, 4, 5), value=1), Dropdown(descripti…

In [9]:
#print(resultados)

In [10]:
#graficador('epsilon-a', mostrar_velocidades=True
           #puntos=puntos_epsilon_a,
           #num_bandas=6,
           #pola='te'#,
           #ejex='Puntos de la rutina (u.i)',
           #ejey=lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)',
           #titulo=lambda num_gap: 'del gap ' + str(num_gap) + ' en función de los puntos de la rutina',
           #param_red=0.63e-6,
           #rango=[0.0,1.0e15]
#          )

In [11]:
#graficador('rutina-num')

In [12]:
#graficador('rutina-num-bandas')

In [13]:
#graficador('rutina-resolucion')

In [14]:
#graficador('rutina-mesh')

In [15]:
#graficador(resultados,
#           puntos=0,
#           num_bandas=6,
#           pola='te',
#           texto=0,
#           param_red=0.0,
#           rango=0.0,
#           mostrar_velocidades=False
#          )

texto_lineal = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                         str(track+1) + " bandas " + pola.upper() + " con e maximo " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
            'vels' : { 'titulo' : lambda track,pola,identidad: 
                                         "Velocidades de grupo para " + str(track+1) + " bandas "
                                          + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
          'rutina' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función del e maximo', 
                         'ejex' : lambda _: 'e maximo (adim.)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
         'widgets' : { 'puntos' : lambda _: 'e maximo',
                      'num_gap' : lambda _: 'Gap',
                        'serie' : lambda _: 'Serie:'
                     }
        }

texto_2mat = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                         str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
            'vels' : { 'titulo' : lambda track,pola,identidad: 
                                         "Velocidades de grupo para " + str(track+1) + " bandas "
                                          + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
          'rutina' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'e grande (adim.)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
         'widgets' : { 'puntos' : lambda _: 'e1',
                      'num_gap' : lambda _: 'Gap',
                        'serie' : lambda _: 'Serie:'
                     }
        }

texto_3mat = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                         str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
            'vels' : { 'titulo' : lambda track,pola,identidad: 
                                         "Velocidades de grupo para " + str(track+1) + " bandas "
                                          + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
          'rutina' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'e grande (adim.)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
         'widgets' : { 'puntos' : lambda _: 'e1',
                      'num_gap' : lambda _: 'Gap',
                        'serie' : lambda _: 'Serie:'
                     }
        }




In [16]:
resultados = graficador('lineal', texto=texto_lineal, num_bandas=6)


interactive(children=(SelectionSlider(description='e maximo', options=(1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8,…

interactive(children=(SelectionSlider(description='Gap', options=(1, 2, 3, 4, 5), value=1), Dropdown(descripti…

In [17]:
#tipo = 'Ancho TE'
#num_gap = 1
#np.max(resultados['gaps'][tipo][num_gap-1])
#np.where(resultados['gaps'][tipo][num_gap-1] == np.max(resultados['gaps'][tipo][num_gap-1]))
#resultados['puntos'][16]

In [18]:
#graficador('lineal-2mat', texto=texto_2mat, num_bandas=6)


In [19]:
#graficador('lineal-3mat', texto=texto_3mat, num_bandas=6)


In [20]:
#graficador('aprox-lineal2-2mat', num_bandas =8)

In [21]:
#graficador('aprox-lineal2-3mat', num_bandas=8)

In [22]:
#graficador('aprox-lineal2-10mat', num_bandas=8)

In [26]:
graficador('para-efectivo', num_bandas=6)

interactive(children=(SelectionSlider(description='Punto de rutina', options=(1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.…

interactive(children=(SelectionSlider(description='Gap', options=(1, 2, 3, 4, 5), value=1), Dropdown(descripti…